# Quantum Computing Algorithms in Cirq

This notebook is the implementation of some of the standard algorthms that are encountered in quantum computing.

The discussion's in here is expanded from examples found in [Cirq examples](https://github.com/quantumlib/Cirq/tree/master/examples) directory.

## Packages

In [1]:
import random
import numpy as np
import cirq

## Quantum Teleportation

Quantum Teleportation is a process by which a quantum state can be transmitted
by sending only two classical bits of information. This is accomplished by
pre-sharing an entangled state between the sender (Alice) and the receiver
(Bob). This entangled state allows the receiver (Bob) of the two classical
bits of information to possess a qubit with the same state as the one held by
the sender (Alice).
In the following example output, qubit 0 (the Message) is set to a random state
by applying X and Y gates. By sending two classical bits of information after
qubit 0 (the Message) and qubit 1 (Alice's entangled qubit) are measured, the
final state of qubit 2 (Bob's entangled qubit) will be identical to the
original random state of qubit 0 (the Message). This is only possible given
that an entangled state is pre-shared between Alice and Bob.

=== REFERENCES ===

https://en.wikipedia.org/wiki/Quantum_teleportation
https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.70.1895

In [2]:
def quantum_teleportation_circuit(gate):
    circuit = cirq.Circuit()
    message, alice, bob = cirq.LineQubit.range(3)
    
    # Creates Bell state to be shared between Alice and Bob
    circuit.append([cirq.H(alice), cirq.CNOT(alice, bob)])
    # Creates a random state for the message
    circuit.append(gate(message))
    # Bell measurement of the Message and Alice's entagled qubit
    circuit.append([cirq.CNOT(message, alice), cirq.H(message)])
    circuit.append(cirq.measure(message, alice))
    # Uses the two classical bits from the Bell measurement to recover
    # the original quantum Message on Bob's entagled qubit
    circuit.append([cirq.CNOT(alice, bob), cirq.CZ(message, bob)])
    
    return circuit

gate = cirq.MatrixGate(cirq.testing.random_unitary(2))
circuit = quantum_teleportation_circuit(gate)

print('Circuit:\n', circuit)

simulation = cirq.Simulator()

# Create qubits
q_0 = cirq.LineQubit(0) 

# Produce the message using random unitary
message = simulation.simulate(cirq.Circuit(gate.on(q_0)))

print('Bloch Vector of Message After Random Unitary: ')

# Prints the Bloch vector of the Message after the random gate
b_0X, b_0Y, b_0Z = cirq.bloch_vector_from_state_vector(message.final_state_vector, 0)

print(
    'x: ', np.around(b_0X, 4),
    'y: ', np.around(b_0Y, 4),
    'z: ', np.around(b_0Z, 4)
)

# Records the final state of the simulation
final_results = simulation.simulate(circuit)

print("Bloch Sphere of Qubit 2 at Final State:")
# Prints the Bloch Sphere of Bob's entangled qubit at the final state
b_2X, b_2Y, b_2Z = cirq.bloch_vector_from_state_vector(final_results.final_state_vector, 2)
print(
    "x: ", np.around(b_2X, 4),
    "y: ", np.around(b_2Y, 4),
    "z: ", np.around(b_2Z, 4)
)

Circuit:
       ┌                           ┐
0: ───│-0.512+0.783j -0.317-0.155j│───────@───H───M───────@───
      │-0.286-0.206j -0.448+0.822j│       │       │       │
      └                           ┘       │       │       │
                                          │       │       │
1: ───H───────────────────────────────@───X───────M───@───┼───
                                      │               │   │
2: ───────────────────────────────────X───────────────X───@───
Bloch Vector of Message After Random Unitary: 
x:  -0.0299 y:  0.6598 z:  0.7508
Bloch Sphere of Qubit 2 at Final State:
x:  -0.0299 y:  0.6598 z:  0.7508


## Deutsch's Algorithm

Deutsch's algorithm is one of the simplest demonstrations of quantum parallelism
and interference. It takes a black-box oracle implementing a Boolean function
f(x), and determines whether f(0) and f(1) have the same parity using just one
query.  This version of Deutsch's algorithm is a simplified and improved version
from Nielsen and Chuang's textbook.

=== REFERENCE ===

https://en.wikipedia.org/wiki/Deutsch–Jozsa_algorithm
Deutsch, David. "Quantum theory, the Church-Turing Principle and the universal
quantum computer." Proc. R. Soc. Lond. A, 400:97, 1985.

In [3]:
def oracle(q_0, q_1, secret_function):
    if secret_function[0]:
        yield [cirq.CNOT(q_0, q_1), cirq.X(q_1)]
        
    if secret_function[1]:
        yield cirq.CNOT(q_0, q_1)

def deutsch_circuit(q_0, q_1, oracle):
    circuit = cirq.Circuit()
    
    # Initialize quibits
    circuit.append([cirq.X(q_1), cirq.H(q_1), cirq.H(q_0)])
    
    # Query oracle
    circuit.append(oracle)
    
    # Measure in X basics
    circuit.append([cirq.H(q_0), cirq.measure(q_0, key='result')])
    
    return circuit

# Choose qubits
q_0, q_1 = cirq.LineQubit.range(2)

# Pick a secret 2-bit function and create a circuit to query the oracle
secret_function = [random.randint(0,1) for _ in range(2)]
oracle = oracle(q_0, q_1, secret_function)

print('Secret function:\nf(x) = <{}>'.format(','.join(str(e) for e in secret_function)))

# Embed the oracle into a quantum circuit querying it exactly once.
circuit = deutsch_circuit(q_0, q_1, oracle)

print('Circuit:\n', circuit)

# Simulate the circuit
simulation = cirq.Simulator()
result = simulation.run(circuit)

print('Result of f(0)⊕f(1):\n', result)

Secret function:
f(x) = <0,0>
Circuit:
 0: ───H───H───M('result')───

1: ───X───H─────────────────
Result of f(0)⊕f(1):
 result=0


## Quantum Fourier Transform and Phase Estimation

###  1. Quantum Fourier Transform
Suppose we have $n$ qubits in the state $|x\rangle$, where $x$ is an integer in the range $0$ to $2^{n-1}$. The quantum Fourier transform (QFT) performs the following operation:
$$
\text{QFT}|x\rangle = \frac{1}{2^{n/2}} \sum_{y=0}^{2^n-1} e^{2\pi i y x/2^n} |y\rangle.
$$
When $x=0$, this is the same as the action of $H^{\otimes n}$ (this is an important sanity check). Though it may not be obvious at first glance, the QFT is actually a unitary transformation. As a matrix, the QFT is given by
$$
\text{QFT} = \begin{bmatrix}
1 & 1 & 1& \cdots &1 \\
1 & \omega & \omega^2& \cdots &\omega^{2^n-1} \\
1 & \omega^2 & \omega^4& \cdots &\omega^{2(2^n-1)}\\
\vdots &\vdots &\vdots &\ddots &\vdots \\
1 &\omega^{2^n-1} &\omega^{2(2^n-1)} &\cdots &\omega^{(2^n-1)(2^n-1)},
\end{bmatrix}
$$
where $\omega = e^{2\pi i /2^n}$. If you believe that the QFT is unitary, then you'll also notice from the matrix form that its inverse is given by a similar expression but with complex-conjugated coefficients:
$$
\text{QFT}^{-1}|x\rangle = \frac{1}{2^{n/2}} \sum_{y=0}^{2^n-1} e^{-2\pi i y x/2^n} |y\rangle.
$$

We just need to understand the notation a little bit. $x_j$ on the left-hand side represents one of the binary digits of the input $x$. $x_1$ is the most significant bit and $x_n$ the least significant bit:
$$
x = \sum_{j=0}^{n-1} x_{j+1}2^j.
$$
$H$ is the Hadamard gate, as usual. The Controlled-$R_j$ gates are phase gates similar to the Controlled-$Z$ gate. In fact, for us it will be useful to just think of them as fractional powers of Controlled-$Z$ gates:
$$
CR_j = CZ^{\large 1/2^{j-1}}
$$
Finally, on the far right we have the output representing the bts of $y$. The only difference between the left and right side is that the output bits are in a different order: the most significant bit of $y$ is on the bottom and the least significant bit is on the top.

#### Quantum Fourier Transform as a Circuit
Let's define a generator which produces the QFT circuit. It should accept a list of qubits as input and `yield`s the gates to construct the QFT in the right order. A useful observation is that the the QFT circuit "repeats" smaller versions of itself as you move from left to right across the diagram.

In [4]:
def make_qft(qubits):
    """
    ---H--@-------@--------@---------------------------------------------
        |       |        |
    ------@^0.5---+--------+---------H--@-------@------------------------
                |        |            |       |
    --------------@^0.25---+------------@^0.5---+---------H--@-----------
                         |                    |            |
    -----------------------@^0.125--------------@^0.25-------@^0.5---H---
    """
    qubits = list(qubits)
    
    while len(qubits) > 0:
        q_head = qubits.pop(0)
        yield cirq.H(q_head)
        
        for i, qubit in enumerate(qubits):
            yield (cirq.CZ**(1/2**(i+1)))(qubit, q_head)
            
num_qubits = 4
qubits = cirq.LineQubit.range(num_qubits)

qft = cirq.Circuit(*make_qft(qubits))
print(qft)

                  ┌───────┐   ┌────────────┐   ┌───────┐
0: ───H───@────────@───────────@───────────────────────────────────────
          │        │           │
1: ───────@^0.5────┼─────H─────┼──────@─────────@──────────────────────
                   │           │      │         │
2: ────────────────@^0.25──────┼──────@^0.5─────┼─────H────@───────────
                               │                │          │
3: ────────────────────────────@^(1/8)──────────@^0.25─────@^0.5───H───
                  └───────┘   └────────────┘   └───────┘


#### Quantum Fourier Transform as a Gate
For later convenience, it will be useful to encapsulate the QFT construction into a single gate. We can inherit from  `cirq.Gate` to define a gate which acts on an unspecified number of qubits, and then use the same strategy as for `make_qft` in the `_decompose_` method of the gate.

In [5]:
class QFT(cirq.Gate):
    def __init__(self, n_qubits):
        self.n_qubits = n_qubits

    def num_qubits(self):
        return self.n_qubits

    def _decompose_(self, qubits):
        """Implements the QFT on an arbitrary number of qubits. The circuit
        for num_qubits = 4 is given by
        ---H--@-------@--------@---------------------------------------------
              |       |        |
        ------@^0.5---+--------+---------H--@-------@------------------------
                      |        |            |       |
        --------------@^0.25---+------------@^0.5---+---------H--@-----------
                               |                    |            |
        -----------------------@^0.125--------------@^0.25-------@^0.5---H---
        """
        qubits = list(qubits)

        while len(qubits) > 0:
            q_head = qubits.pop(0)
            yield cirq.H(q_head)

            for i, qubit in enumerate(qubits):
                yield (cirq.CZ**(1/2**(i+1)))(qubit, q_head)

    def _circuit_diagram_info_(self, args):
        return tuple('QFT{}'.format(i) for i in range(self.n_qubits))
    
# Test the Circuit
num_qubits = 4

qubits = cirq.LineQubit.range(num_qubits)
circuit = cirq.Circuit(QFT(num_qubits).on(*qubits))
print(circuit)

qft_test = cirq.Circuit(*make_qft(qubits))
print(qft_test)
np.testing.assert_allclose(cirq.unitary(qft_test), cirq.unitary(circuit))

0: ───QFT0───
      │
1: ───QFT1───
      │
2: ───QFT2───
      │
3: ───QFT3───
                  ┌───────┐   ┌────────────┐   ┌───────┐
0: ───H───@────────@───────────@───────────────────────────────────────
          │        │           │
1: ───────@^0.5────┼─────H─────┼──────@─────────@──────────────────────
                   │           │      │         │
2: ────────────────@^0.25──────┼──────@^0.5─────┼─────H────@───────────
                               │                │          │
3: ────────────────────────────@^(1/8)──────────@^0.25─────@^0.5───H───
                  └───────┘   └────────────┘   └───────┘


#### Inverse QFT

In [6]:
class QFT_inv(cirq.Gate):
    def __init__(self, n_qubits):
        self.n_qubits = n_qubits

    def num_qubits(self):
        return self.n_qubits

    def _decompose_(self, qubits):
        """Implements the inverse QFT on an arbitrary number of qubits. The circuit
        for num_qubits = 4 is given by
        ---H--@-------@--------@---------------------------------------------
              |       |        |
        ------@^-0.5--+--------+---------H--@-------@------------------------
                      |        |            |       |
        --------------@^-0.25--+------------@^-0.5--+---------H--@-----------
                               |                    |            |
        -----------------------@^-0.125-------------@^-0.25------@^-0.5--H---
        """
        qubits = list(qubits)
        while len(qubits) > 0:
            q_head = qubits.pop(0)
            yield cirq.H(q_head)
            for i, qubit in enumerate(qubits):
                yield (cirq.CZ**(-1/2**(i+1)))(qubit, q_head)

    def _circuit_diagram_info_(self, args):
        return tuple('QFT{}^-1'.format(i) for i in range(self.n_qubits))
    
num_qubits = 2

qubits = cirq.LineQubit.range(num_qubits)
circuit = cirq.Circuit(QFT(num_qubits).on(*qubits),
                                QFT_inv(num_qubits).on(*qubits))
print(circuit)
cirq.unitary(circuit).round(2)

0: ───QFT0───QFT0^-1───
      │      │
1: ───QFT1───QFT1^-1───


array([[ 1. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ],
       [-0. +0.j ,  0.5+0.5j, -0. +0.j ,  0.5-0.5j],
       [ 0. +0.j ,  0.5+0.j ,  0.5-0.5j,  0. +0.5j],
       [-0. +0.j ,  0. -0.5j,  0.5+0.5j,  0.5+0.j ]])

If `QFT` and `QFT_inv` were really inverses then we would have gotten the identity matrix here. There are a couple of ways to fix this. One is to change the implementations of these two gates in such a way that the outputs are "rightside-up." A different solution is to turn the qubits around in between acting with `QFT` and `QFT_inv`. In other words, we can insert the `QFT_inv` gate "upside-down" as follows:

In [7]:
num_qubits = 2

qubits = cirq.LineQubit.range(num_qubits)
circuit = cirq.Circuit(QFT(num_qubits).on(*qubits),
                                QFT_inv(num_qubits).on(*qubits[::-1])) # qubit order reversed
print(circuit)
cirq.unitary(circuit)

0: ───QFT0───QFT1^-1───
      │      │
1: ───QFT1───QFT0^-1───


array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
       [-0.+0.j, -0.+0.j,  1.+0.j, -0.+0.j],
       [-0.+0.j, -0.+0.j, -0.+0.j,  1.+0.j]])

We find the identity matrix, as desired (up to finite-precision numerical errors). Notice that the `QFT_inv` gate is upside-down relative to the `QFT` gate in the diagram. This is why we included the extra digits in the `wire_symobls`!

### 2. Phase Estimation
As an application of our quantum Fourier transform circuit, we'll implement the [phase estimation](https://en.wikipedia.org/wiki/Quantum_phase_estimation_algorithm) algorithm. The phase estimation algorithm uses the inverse QFT to give an estimate of the eigenvalue of a unitary operator.

Suppose we have a unitary operator $U$ with eigenvactor $|\psi\rangle$ and eigenvalue $\exp(2\pi i \theta)$. Our objective is to get an $n$-bit approximation to $\theta$. The first step is to construct the state
$$
|\Phi\rangle = \frac{1}{2^{n/2}}\sum_{y=0}^{2^{n-1}} e^{2\pi i y \theta}|y\rangle.
$$
This looks very similar to the output of the QFT applied to the state $|2^n\theta\rangle$, except for the fact that $2^n\theta$ may not be an integer. If $2^n\theta$ *were* an integer, then we would apply the inverse QFT and measure the qubits to read off the binary representation of $2^n\theta$. Even if $2^n\theta$ is not an integer, we can still perform the same procedure and the result will be a sequence of bits that, with high probility, gives an $n$-bit approximation to $\theta$. We just have to repeat the procedure a few times to be sure of the answer.

Since we've already constructed the inverse QFT, all we really have to do is figure out how to construct the magic state $|\Phi\rangle$. This is accomplished by the first part of the circuit picutred above. We begin by applying $H^{\otimes n}$ to the state $|0\rangle$, creating an equal superposition over all basis states:
$$
H^{\otimes n} |0\rangle = \frac{1}{2^{n/2}}\sum_{y=0}^{2^n-1}|y\rangle.
$$
Now we need to insert the correct phase coefficients. This is done by a sequence of Controlled-$U^k$ operations, where the qubits of $y$ are the controls and the $U^k$ operations act on $|\psi \rangle$.

Let's try to implement this part of the procedure in Cirq, and then put it together with our `QFT_inv` from above. For the gate $U$ we'll pick the single-qubit operation
$$
U = Z^{2\theta} = \begin{bmatrix}
1 & 0 \\
0 & e^{2\pi i \theta }
\end{bmatrix}
$$
for $\theta \in [0,1)$. This is just for simplicity and ease of testing. You are invited to write an implementation that accepts an arbitrary $U$.

In [8]:
theta = 0.356
n_bits = 5

qubits = cirq.LineQubit.range(n_bits)
u_bit = cirq.NamedQubit('u')

U = cirq.Z**(2*theta)

phase_estimator = cirq.Circuit()

phase_estimator.append(cirq.H.on_each(*qubits))

for i, bit in enumerate(qubits):
    phase_estimator.append(cirq.ControlledGate(U).on(bit,u_bit)**(2**(n_bits-1-i)))

print(phase_estimator)

0: ───H───@────────────────────────────────────────────────────
          │
1: ───H───┼──────────@─────────────────────────────────────────
          │          │
2: ───H───┼──────────┼──────────@──────────────────────────────
          │          │          │
3: ───H───┼──────────┼──────────┼─────────@────────────────────
          │          │          │         │
4: ───H───┼──────────┼──────────┼─────────┼──────────@─────────
          │          │          │         │          │
u: ───────Z^-0.608───Z^-0.304───Z^0.848───Z^-0.576───Z^0.712───


The next step is to perform the inverse QFT on estimation qubits:

In [9]:
phase_estimator.append(QFT_inv(n_bits).on(*qubits))
print(phase_estimator)

0: ───H───@────────────────────────────────────────────────────QFT0^-1───
          │                                                    │
1: ───H───┼──────────@─────────────────────────────────────────QFT1^-1───
          │          │                                         │
2: ───H───┼──────────┼──────────@──────────────────────────────QFT2^-1───
          │          │          │                              │
3: ───H───┼──────────┼──────────┼─────────@────────────────────QFT3^-1───
          │          │          │         │                    │
4: ───H───┼──────────┼──────────┼─────────┼──────────@─────────QFT4^-1───
          │          │          │         │          │
u: ───────Z^-0.608───Z^-0.304───Z^0.848───Z^-0.576───Z^0.712─────────────


At this point we are almost ready to measure the estimation qubits. But we should also specify an initial state for the `u_bit`. By default it will be the state $|0\rangle$, but the phase for that state is trivial with the operator we chose. Inserting a Pauli $X$ operator at the begining of the circuit changes this to the $|1\rangle$ state, which has the nontrivial $\theta$ phase.

In [10]:
# Add measurements to the end of the circuit
phase_estimator.append(cirq.measure(*qubits, key='m'))

# Add gate to change initial state to |1>
phase_estimator.insert(0,cirq.X(u_bit))

print(phase_estimator)

0: ───H───@────────────────────────────────────────────────────QFT0^-1───M('m')───
          │                                                    │         │
1: ───H───┼──────────@─────────────────────────────────────────QFT1^-1───M────────
          │          │                                         │         │
2: ───H───┼──────────┼──────────@──────────────────────────────QFT2^-1───M────────
          │          │          │                              │         │
3: ───H───┼──────────┼──────────┼─────────@────────────────────QFT3^-1───M────────
          │          │          │         │                    │         │
4: ───H───┼──────────┼──────────┼─────────┼──────────@─────────QFT4^-1───M────────
          │          │          │         │          │
u: ───X───Z^-0.608───Z^-0.304───Z^0.848───Z^-0.576───Z^0.712──────────────────────


Now we can intstantiate a simulator and make measurements of the estimation qubits. Let the values of these qubits be $a_j$. Then our $n$-bit approximation for $\theta$ is given by
$$
\theta \approx \sum_{j=0}^n a_j2^{-j}.
$$
We'll perform this conversion from bit values to $\theta$-values and then print the results:

In [11]:
simulation = cirq.Simulator()
result = simulation.run(phase_estimator, repetitions=10)

theta_estimates = np.sum(2**np.arange(n_bits)*result.measurements['m'], axis=1)/2**n_bits
print(theta_estimates)

[0.375   0.34375 0.875   0.375   0.34375 0.3125  0.34375 0.375   0.375
 0.34375]


In [12]:
def phase_estimation(theta, n_bits, n_reps=10):
    qubits = cirq.LineQubit.range(n_bits)
    u_bit = cirq.NamedQubit('u')

    U = cirq.Z**(2*theta) # Try out a different gate if you like

    phase_estimator = cirq.Circuit()

    phase_estimator.append(cirq.H.on_each(*qubits))
    
    for i, bit in enumerate(qubits):
        phase_estimator.append(cirq.ControlledGate(U).on(bit,u_bit)**(2**(n_bits-1-i)))
        
    phase_estimator.append(QFT_inv(n_bits).on(*qubits))

    # Measurement gates
    phase_estimator.append(cirq.measure(*qubits, key='m'))

    # Gates to choose initial state for the u_bit. Placing X here chooses the |1> state
    phase_estimator.insert(0,cirq.X(u_bit))

    # Code to simulate measurements
    simulation = cirq.Simulator()
    result = simulation.run(phase_estimator, repetitions=n_reps)

    # Convert measurements into estimates of theta
    theta_estimates = np.sum(2**np.arange(n_bits)*result.measurements['m'], axis=1)/2**n_bits

    return theta_estimates

In [13]:
phase_estimation(0.356, 10)

array([0.35644531, 0.35546875, 0.35644531, 0.35546875, 0.35644531,
       0.35644531, 0.35644531, 0.35644531, 0.35546875, 0.35644531])

#### Phase Estimation Without an Eigenstate

We can modify our `phase_estimation` method to explore other interesting aspects of the algorithm. Suppose the input to the circuit was not an eigenstate of $U$ at all? You could modify the case we analyzed above by putting an arbitrary single-qubit state on the `u_bit`, or you could replace $Z^{2\theta}$ with something like $X^{2\theta}$ while still passing in a computational basis state.

Suppose the operator $U$ has eigenstate $|u_j\rangle$ with eigenvalues $e^{2\pi i \theta_j}$. Then in general if we pass in a state of the form
$$
\sum_j \alpha_j|u_j\rangle
$$
then each time we run the circuit we will get an $n$-bit estimate of one of the $\theta_j$ chosen at random, and the probability of choosing a particular $\theta_j$ is given by $|\alpha_j|^2$. One simple test of this is to modify our above code to pass the state
$$
\frac{|0\rangle + |1\rangle}{\sqrt{2}}
$$
into the phase estimator for $Z^{2\theta}$. The state $|1\rangle$ has eigenvalue $e^{2\pi i \theta_j}$ while the state $|0\rangle$ has eigenvalue $1$.

In [14]:
def phase_estimation(theta, n_bits, n_reps=10):
    qubits = cirq.LineQubit.range(n_bits)
    u_bit = cirq.NamedQubit('u')

    U = cirq.Z**(2*theta)

    phase_estimator = cirq.Circuit()

    phase_estimator.append(cirq.H.on_each(*qubits))
    
    for i, bit in enumerate(qubits):
        phase_estimator.append(cirq.ControlledGate(U).on(bit,u_bit)**(2**(n_bits-1-i)))
        
    phase_estimator.append(QFT_inv(n_bits).on(*qubits))

    phase_estimator.append(cirq.measure(*qubits, key='m'))

    # Changed the X gate here to an H
    phase_estimator.insert(0,cirq.H(u_bit))

    simulation = cirq.Simulator()
    result = simulation.run(phase_estimator, repetitions=n_reps)

    theta_estimates = np.sum(2**np.arange(n_bits)*result.measurements['m'], axis=1)/2**n_bits

    return theta_estimates

In [15]:
phase_estimation(0.356,10)

array([0.35546875, 0.        , 0.        , 0.        , 0.35644531,
       0.35644531, 0.        , 0.        , 0.35449219, 0.35644531])

### 3. Grover's Search Algorithm

The Grover algorithm takes a black-box oracle implementing a function
$f(x) = 1\text{ if }x=x',~ f(x) = 0 \text{ if } x\neq x'$ and finds $x'$ within a randomly ordered sequence of $N$ items using $O(\sqrt{N}$) operations and $O(N\,  \text{log}N$) gates,
with the probability $p \geq 2/3$.
At the moment, only 2-bit sequences (for which one pass through Grover operator
is enough) are considered.

=== REFERENCE ===

Coles, Eidenbenz et al. Quantum Algorithm Implementations for Beginners
https://arxiv.org/abs/1804.03719

In [16]:
def set_io_qubits(qubit_count):
    input_qubits = [cirq.GridQubit(i, 0) for i in range(qubit_count)]
    output_qubit = cirq.GridQubit(qubit_count, 0)
    return (input_qubits, output_qubit)


def oracle(input_qubits, output_qubit, x_bits):
    # Make oracle.
    # for (1, 1) it's just a Toffoli gate
    # otherwise negate the zero-bits.
    yield(cirq.X(q) for (q, bit) in zip(input_qubits, x_bits) if not bit)
    yield(cirq.TOFFOLI(input_qubits[0], input_qubits[1], output_qubit))
    yield(cirq.X(q) for (q, bit) in zip(input_qubits, x_bits) if not bit)


def grover_circuit(input_qubits, output_qubit, oracle):
    # For 2 input qubits, that means using Grover operator only once.
    circuit = cirq.Circuit()

    # Initialize qubits.
    circuit.append([
        cirq.X(output_qubit),
        cirq.H(output_qubit),
        cirq.H.on_each(*input_qubits),
    ])

    # Query oracle.
    circuit.append(oracle)

    # Construct Grover operator.
    circuit.append(cirq.H.on_each(*input_qubits))
    circuit.append(cirq.X.on_each(*input_qubits))
    circuit.append(cirq.H.on(input_qubits[1]))
    circuit.append(cirq.CNOT(input_qubits[0], input_qubits[1]))
    circuit.append(cirq.H.on(input_qubits[1]))
    circuit.append(cirq.X.on_each(*input_qubits))
    circuit.append(cirq.H.on_each(*input_qubits))

    # Measure the result.
    circuit.append(cirq.measure(*input_qubits, key='result'))

    return circuit


def bitstring(bits):
    return ''.join(str(int(b)) for b in bits)


qubit_count = 2
circuit_sample_count = 10

#Set up input and output qubits.
(input_qubits, output_qubit) = set_io_qubits(qubit_count)

#Choose the x' and make an oracle which can recognize it.
x_bits = [random.randint(0, 1) for _ in range(qubit_count)]
print('Secret bit sequence: {}'.format(x_bits))

# Make oracle (black box)
oracle = oracle(input_qubits, output_qubit, x_bits)

# Embed the oracle into a quantum circuit implementing Grover's algorithm.
circuit = grover_circuit(input_qubits, output_qubit, oracle)
print('Circuit:')
print(circuit)

# Sample from the circuit a couple times.
simulation = cirq.Simulator()
result = simulation.run(circuit, repetitions=circuit_sample_count)

frequencies = result.histogram(key='result', fold_func=bitstring)
print('Sampled results:\n{}'.format(frequencies))

# Check if we actually found the secret value.
most_common_bitstring = frequencies.most_common(1)[0][0]
print('Most common bitstring: {}'.format(most_common_bitstring))
print('Found a match: {}'.format(
    most_common_bitstring == bitstring(x_bits)))

Secret bit sequence: [0, 0]
Circuit:
(0, 0): ───H───X───@───X───H───X───────@───X───H───────M('result')───
                   │                   │               │
(1, 0): ───H───X───@───X───H───X───H───X───H───X───H───M─────────────
                   │
(2, 0): ───X───H───X─────────────────────────────────────────────────
Sampled results:
Counter({'00': 10})
Most common bitstring: 00
Found a match: True


### 4. Brenstein-Vazirani Algorithm

Bernstein and Varizani have given the first quantum algorithm to solve parity problem in which a strong violation of the classical imformation theoritic bound comes about.

=== REFERENCE ===
 J Du et al. Implementation of a quantum algorithm to solve Bernstein-Vazirani's parity problem without entanglement on an ensemble quantum computer
 
https://arxiv.org/abs/quant-ph/0012114

In [17]:
def bitstring(bits):
    return  ''.join(str(int(b)) for b in bits)

def oracle(input_qubits, output_qubit, secret_factor_bits, secret_bias_bit):
    if secret_bias_bit:
        yield cirq.X(output_qubit)
        
    for qubit, bit in zip(input_qubits, secret_factor_bits):
        if bit:
            yield cirq.CNOT(qubit, output_qubit)

def bernstein_vazirani_circuit(input_qubits, output_qubit, oracle):
    circuit = cirq.Circuit()
    
    # Initialize qubits
    circuit.append([
        cirq.X(output_qubit),
        cirq.H(output_qubit),
        cirq.H.on_each(*input_qubits),
    ])
    
    # Query oracle
    circuit.append(oracle)
    
    # Measure in X basis
    circuit.append([
        cirq.H.on_each(*input_qubits),
        cirq.measure(*input_qubits, key='result')
    ])
    
    return circuit

# Number of qubits
qubit_count = 8

# Number of times to sample from the circuit
circuit_sample_count = 3

# Choose qubits to use
input_qubits = [cirq.GridQubit(i, 0) for i in range(qubit_count)]
output_qubit = cirq.GridQubit(qubit_count, 0)

# Pick coefficients for the oracle and create a circuit to query it
secret_bias_bit = random.randint(0, 1)
secret_factor_bits = [random.randint(0, 1) for _ in range(qubit_count)]

oracle = oracle(input_qubits, output_qubit, secret_factor_bits, secret_bias_bit)
print('Secret function:\nf(x) = x*<{}> + {} (mod 2)'.format(','.join(str(e) for e in secret_factor_bits), secret_bias_bit))

# Embed the oracle into a special quantum circuit querying it exactly once
circuit = bernstein_vazirani_circuit(input_qubits, output_qubit, oracle)
print('\nCircuit:\n', circuit)

# Sample from the circuit a couple times
simulation = cirq.Simulator()
result = simulation.run(circuit, repetitions=circuit_sample_count)
frequencies = result.histogram(key='result', fold_func=bitstring)

print('\nSampled results:\n{}'.format(frequencies))

# Check if we actually found the secret value:
most_common_bitstring = frequencies.most_common(1)[0][0]
print('\nMost common matches secret factors:\n{}'.format(most_common_bitstring == bitstring(secret_factor_bits)))

Secret function:
f(x) = x*<0,0,1,1,1,1,0,0> + 0 (mod 2)

Circuit:
 (0, 0): ───H───H───────────────────────M('result')───
                                       │
(1, 0): ───H───H───────────────────────M─────────────
                                       │
(2, 0): ───H───────@───H───────────────M─────────────
                   │                   │
(3, 0): ───H───────┼───@───H───────────M─────────────
                   │   │               │
(4, 0): ───H───────┼───┼───@───H───────M─────────────
                   │   │   │           │
(5, 0): ───H───────┼───┼───┼───@───H───M─────────────
                   │   │   │   │       │
(6, 0): ───H───H───┼───┼───┼───┼───────M─────────────
                   │   │   │   │       │
(7, 0): ───H───H───┼───┼───┼───┼───────M─────────────
                   │   │   │   │
(8, 0): ───X───H───X───X───X───X─────────────────────

Sampled results:
Counter({'00111100': 3})

Most common matches secret factors:
True
